In [2]:
import Extractor
import pandas as pd
import sqlite3

conn1 = sqlite3.connect('Humor.db')
cur1 = conn1.cursor()
df1 = pd.read_sql('SELECT * FROM head ORDER BY wdate DESC',conn1)
conn1.close()

conn2 = sqlite3.connect('Politic.db')
cur2 = conn2.cursor()
df2 = pd.read_sql('SELECT * FROM head ORDER BY wdate DESC',conn2)
conn2.close()

df = df1.append(df2) # 엔터테인먼트는 여기서 분할해서 저장해야함.

df['pk'] = range(1, len(df)+1)

conn3 = sqlite3.connect('HP.db')
cur3 = conn3.cursor()
cur3.executescript('''
            DROP TABLE IF EXISTS head;
            CREATE TABLE head(
                pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                head TEXT NOT NULL,
                wdate TEXT NOT NULL,
                cdate TEXT NOT NULL,
                ref INTEGER NOT NULL,
                page INTEGER NOT NULL
            );
        ''')
df.to_sql('head', conn3, if_exists='append', index=False)
conn3.close()

In [ ]:
import Extractor
import pandas as pd
import sqlite3

conn = sqlite3.connect('HP.db')
cur = conn.cursor()
df = pd.read_sql('SELECT head FROM head ORDER BY wdate DESC',conn)
conn.close()
df.drop_duplicates(keep='first', inplace=True)
ext = Extractor.Ext(df)
df = ext.cleaning()

new_words = ext.search_dict(sorted(ext.extract_nouns().items(),key=lambda _:_[1], reverse=True))
sent = ext.extract_sent(new_words)

# 변수 생성
statistic = ext.extract_statistic_value(sent)
r_rat = ext.extract_r_rat(sent,statistic)
statistic = ext.combine_var(statistic, r_rat)
conn = sqlite3.connect('HP_var.db')
statistic.to_sql('var', conn)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 1410442 from 1832364 sents. mem=0.865 Gb                    
